In [38]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from PythonCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    
    # Logo
    html.Div(
        html.Img(
            src=f'data:image/png;base64,{encoded_image.decode()}' if encoded_image else '',
            style={
                'display': 'block',          # Ensures the image is treated as a block element
                'margin-left': 'auto',       # Centers the image horizontally
                'margin-right': 'auto',      # Centers the image horizontally
                'width': '300px',            # Adjusts the width of the image
                'height': 'auto'             # Maintains the aspect ratio
            }
        )
    ),
    
    # Branding
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Riley Johnson', 
                              style={
                                'font-size': '20px',  # Adjust size in pixels
                                }
                              ))),
    html.Hr(),
    
    # Radio items 
    html.Div([
        html.Center(html.Label(['Filter table by rescue dog types'], style={'font-weight': 'bold'})),
        html.Center(dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'Filter by Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Filter by Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Filter by Disaster Rescue'},
                {'label': 'View All Animals', 'value': 'Remove all filters'}
            ],
            value='Remove all filters', 
            style={
            'margin-top': '20px', 
            'margin-bottom': '20px'
            },
            inputStyle={"margin-left": "20px"},
            labelStyle={'display': 'inline-block'}
        ))
    ]),
    
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         
#Set up the features for your interactive data table to make it user-friendly for your client
        row_selectable="single",
        selected_rows=[0],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10
                        ),
                    
        html.Br(),
        html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
             style={'display': 'flex', "width": "500"},
             children=[
                 html.Div(
                     style={'display': 'flex'},
                     children=[
                         # pie chart
                         dcc.Graph(
                             id='graph-id',
                         ),
                         # map
                         html.Div(
                             id='map-id',
                             className='col s12 m6',
                         ),
                     ]
                 ),
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update datatable 
@app.callback(Output('datatable-id','data'),
              [Input(component_id='radio-items', component_property='value')])

def update_dashboard(radio_options):

    # Default empty dataframe
    dff = pd.DataFrame()
    query = {}
    
    try:
        if radio_options == 'Filter by Water Rescue':
            query = {
            'animal_type': 'Dog',
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
            }

        elif radio_options == 'Filter by Mountain Rescue':
            query = {
            'animal_type': 'Dog',
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
            }

        elif radio_options == 'Filter by Disaster Rescue':
            query = {
            'animal_type': 'Dog',
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
            }

        # Fetch data from database
        data = db.read(query)
        if not data:
            print("No data retrieved from the database.")
            return []

        dff = pd.DataFrame(data)
        
        if '_id' in dff.columns:
            dff.drop(['_id'], axis=1, inplace=True)
            
    except Exception as e:
        print(f"Error in update_dashboard: {e}")

    # Return data or empty message
    if dff.empty:
        return []
    
    return dff.to_dict('records')

    # Debug: Check DataFrame columns
    #print("Filtered DataFrame Columns:", dff.columns)
    #print("Filtered DataFrame Head:", dff.head())


# additional graph - pie chart 
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    
    # Initialize an empty dataframe
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    
    # Debug: Print columns
    #print("Columns in DataFrame:", dff.columns)
    
    # Handle missing 'breed' column
    if dff.empty or 'breed' not in dff.columns:
        print("Warning: 'breed' column missing or DataFrame is empty.")
        return px.pie(data_frame=pd.DataFrame(), title="No Data Available")
    
    # breeds to include
    pie_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever','Newfoundland', 
                  'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 
                  'Siberian Husky', 'Rottweiler', 'Doberman Pinscher', 'Golden Retriever',
                  'Bloodhound']
    
    # filter dataframe 
    dff = dff[dff['breed'].isin(pie_breeds)]

    # Create the pie chart
    piechart = px.pie(
        data_frame=dff,
        names='breed',
        hole=0.3,
    )
    
    # Update layout to remove percentages/labels
    piechart.update_traces(
        textinfo='none'  
    )
    
    # Add a title to the chart
    piechart.update_layout(
        title="Possible Rescue Dogs - Breed Distribution",
        title_font_size=20,  # Adjust font size
        title_x=0.75,  # Center the title
    )
    
    return piechart
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    
    # Return an empty list if no columns are selected
    if not selected_columns:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index): 
    
    if index is None or not index:
        return html.Div("Select a row to see geolocation data.")
    
    # Code for geolocation chart
    dff = df if viewData is None else pd.DataFrame(viewData)

     # Because we only allow single row selection, the list can 
     # be converted to a row index here
    
    # Validate the presence of required columns
    required_columns = ['location_lat', 'location_long', 'breed', 'name']
    if not all(col in dff.columns for col in required_columns):
        return html.Div("Select a row to see geolocation data.")
    
    # Extract Latitude and Longitude as scalars
    lat = dff.iloc[index]['location_lat']
    lon = dff.iloc[index]['location_long']
    
    # Ensure lat and lon are scalars
    if isinstance(lat, pd.Series):
        lat = lat.values[0]
    if isinstance(lon, pd.Series):
        lon = lon.values[0]
    
    # Debugging
    #print("Latitude:", lat)
    #print("Longitude:", lon)
    
    # Validate Latitude and Longitude for the selected row
    if pd.isna(lat) or pd.isna(lon):
        return html.Div("Invalid geolocation data for the selected entry.")

    if index is None:
       row = 0
    else: 
       row = index[0]

     # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
        center=[30.75,-97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and popup
        # Column 13 and 14 define the grid-coordinates for 
        # the map
        # Column 4 defines the breed for the animal
        # Column 9 defines the name of the animal
        dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
        children=[
            dl.Tooltip(dff.iloc[row,4]),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
                ])
            ])
       ])
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:29155/
